In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
from os import listdir
import numpy as np
from tqdm import tqdm

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVD.csv')
df_test = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Test_SurSampling1.csv")
df_train = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Train_SurSampling1.csv")
df_valid = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Val_SurSampling1.csv")
len(df_test), len(df_train), len(df_valid)

(640, 1909, 639)

In [26]:
mat_train = np.load('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Video_Embeddings/ViViTtrain.npy')

In [27]:
mat_train = np.concatenate((np.max(mat_train[:785], axis = 1), np.max(mat_train[785:(785*2)], axis = 1), np.max(mat_train[(785*2):(785*3)], axis = 1), np.max(mat_train[(785*3):], axis = 1))).reshape(len(mat_train), -1)

In [28]:
mat_valid = np.load('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Video_Embeddings/ViViTvalid.npy')

In [29]:
mat_valid = np.concatenate((np.max(mat_valid[:785], axis = 1), np.max(mat_valid[785:(785*2)], axis = 1), np.max(mat_valid[(785*2):(785*3)], axis = 1), np.max(mat_valid[(785*3):], axis = 1)), axis = 0).reshape(len(mat_valid), -1)

In [12]:
dic_names_indices = {}
for count, name in enumerate(df['name']):
  dic_names_indices[name] = count

In [13]:
df_train_indices = np.zeros(len(df_train), np.int16)
df_valid_indices = np.zeros(len(df_valid), np.int16)
df_test_indices = np.zeros(len(df_test), np.int16)
for count, name in enumerate(df_train['name']):
  df_train_indices[count] = dic_names_indices[name]
for count, name in enumerate(df_test['name']):
  df_test_indices[count]  = dic_names_indices[name]
for count, name in enumerate(df_valid['name']):
  df_valid_indices[count] = dic_names_indices[name]
df_train_indices = np.sort(df_train_indices)
df_test_indices = np.sort(df_test_indices)
df_valid_indices = np.sort(df_valid_indices)

In [14]:
label_indice_train = np.unique(df_train_indices)
label_indice_valid = np.unique(df_valid_indices)
label_indice_test = np.unique(df_test_indices)

In [15]:
label_dummies = pd.get_dummies(df['label'])
label_train = label_dummies.loc[label_indice_train].reset_index()
label_valid = label_dummies.loc[label_indice_valid].reset_index()

# SVC

In [16]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(mat_train,df['label'][df_train_indices])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [17]:
tfsvmpred = clf.predict(mat_valid)
accuracy_score(df['label'][df_valid_indices],tfsvmpred)

0.3599374021909233

In [18]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf', 'sigmoid'), 'C':[1, 10], 'gamma': ['scale', 'auto']}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit( mat_train,df['label'][df_train_indices] )
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.741560,0.065260,0.139875,0.011096,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",0.376963,0.390052,0.450262,0.410995,0.377953,0.401245,0.027405,3
1,0.869273,0.034301,0.343212,0.013015,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.327225,0.387435,0.421466,0.384817,0.322835,0.368755,0.037997,8
2,1.156123,0.144591,0.214105,0.023013,1,scale,sigmoid,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.243455,0.230366,0.230366,0.230366,0.225722,0.232055,0.005977,9
3,0.658580,0.015767,0.132931,0.002298,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.376963,0.390052,0.450262,0.410995,0.377953,0.401245,0.027405,3
4,0.868371,0.078649,0.391040,0.060525,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.337696,0.410995,0.434555,0.390052,0.338583,0.382376,0.038768,7
5,1.101935,0.114238,0.210819,0.011329,1,auto,sigmoid,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.243455,0.219895,0.232984,0.230366,0.223097,0.229960,0.008244,10
6,0.675630,0.033697,0.137920,0.006498,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.376963,0.390052,0.450262,0.410995,0.377953,0.401245,0.027405,3
7,0.992769,0.119877,0.357203,0.077369,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.397906,0.431937,0.494764,0.437173,0.385827,0.429521,0.038026,2
8,0.765483,0.015029,0.186326,0.007121,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.172775,0.183246,0.206806,0.201571,0.191601,0.191200,0.012280,12
9,0.662010,0.021602,0.131385,0.004772,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.376963,0.390052,0.450262,0.410995,0.377953,0.401245,0.027405,3


In [19]:
tfsvmpred = clf.predict(mat_valid)
accuracy_score(df['label'][df_valid_indices],tfsvmpred)

0.3677621283255086

In [20]:
tfsvmpred = clf.predict(mat_valid)
accuracy_score(df['label'][df_valid_indices],tfsvmpred)

0.3677621283255086

# Deep Learning

In [21]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Reshape, Input, Add
from tensorflow.keras.layers import ReLU,BatchNormalization
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical

In [22]:
def Direct_3137_768_to_label(dropout = True):
  x_in = Input(768*4)
  x = x_in
  #x = BatchNormalization()(x)
  layers=[1024*4, 2048, 512]
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = ReLU()(x)
    if dropout:
      x = Dropout(0.5)(x)
  x_out = Dense(15, activation = 'sigmoid')(x)
  model = Model(x_in, x_out)
  model.compile(loss = 'categorical_crossentropy', optimizer = Adam())
  return model

model = Direct_3137_768_to_label()
#model.summary()

In [23]:
batch_size = 128
epochs=10
hist = model.fit(mat_train, np.array(label_dummies.loc[df_train_indices]),
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(mat_valid,  np.array(label_dummies.loc[df_valid_indices])))

Epoch 1/10


ValueError: ignored

In [ ]:
model.evaluate(emb_VGG_av_test, emb_MPnet_test)

In [ ]:
import gc
del train_mat
gc.collect()